Set parameters

In [1]:
import numpy as np
import math

In [2]:
T = 5
a = 0.2
sigma = 0.02
phi = 0.05
r_0 = 0.05

In [3]:
N = 50

Set a Class

In [4]:
delta_t = T / N
delta_x = sigma * (3 * delta_t) ** (1 / 2)

delta_t, delta_x

(0.1, 0.010954451150103324)

In [5]:
I = math.ceil(6 ** (1 / 2) / (3 * a * delta_t))
I

41

In [6]:
X = np.zeros((I * 2 + 1, N))
X.shape

(83, 50)

In [7]:
for j in range(N):
    for k in range(min([j + 1, I + 1])):
        # 表からはみ出ないようにしつつ、値を代入
        X[I - k, j] = k * delta_x
        X[I + k, j] = - k * delta_x

In [8]:
X[I - j - 1, j]

-0.36149688795340973

In [15]:
class HullWhite_grid:
    """
    af
    """
    def __init__(self, T, a, sigma, phi, r_0, N):
        """
        初期化する。
        Augs
            T: 最大の時間
            a: ドリフト
        """
        # 入力を整理
        self.T = T
        self.a = a
        self.sigma = sigma
        self.phi = phi
        self.r_0 = r_0
        self.N = N  

        # 格子の縦・横の幅を設定
        self.delta_t = self.T / self.N
        self.delta_x = self.sigma * (3 * self.delta_t) ** (1 / 2)

        # 格子の端点を設定
        self.I = math.ceil(6 ** (1 / 2) / (3 * self.a * self.delta_t))
        
        # 諸々の格子を作成する
        self.X_matrix = self._make_X_matrix()
        self.r_matrix = self._make_r_matrix()
        self.prob_matrix = self._make_prob_matrix()
        
    def calc_discount_bond_price(self, payoff = 1):
        """
        backwardで割引債価格を求める
        Augs
            payoff: 満期における割引債価格のペイオフ(指定しなければ1)
        """
        discount_bond_matrix = np.zeros((self.I * 2 + 1, self.N))
        discount_bond_matrix[:, self.N - 1] = 1
        return discount_bond_matrix
    
    def _make_X_matrix(self):
        """
        Xの格子を作成する
        (I * 2 + 1) × Nの配列をベースに作成
        """
        X_matrix = np.zeros((self.I * 2 + 1, self.N))
        for j in range(self.N):
            # j：列番号
            for i in range(min([j + 1, self.I + 1])):
                # i: 中心線からの距離
                # 表からはみ出ないようにしつつ、値を代入
                X_matrix[self.I - i, j] = k * self.delta_x
                X_matrix[self.I + i, j] = - k * self.delta_x

        return X_matrix
    
    def _theta_function(self, t):
        theta = self.phi / self.a * (1 - math.exp(- self.a * t)) + self.r_0 * math.exp(- self.a * t)        
        return theta
    
    def _make_r_matrix(self):
        """
        スポットレートの格子を作る
        X_matrixをtheta(t)分シフトするだけ
        """
        r_matrix = self.X_matrix.copy()
        
        for j in range(self.N):
            # j：列番号
            r_matrix[self.I, j] += self._theta_function(j)
            for i in range(1, min([j + 1, self.I + 1])):
                # i: 中心線からの距離
                # 表からはみ出ないようにしつつ、値を加算
                r_matrix[self.I - i, j] += self._theta_function(j)
                r_matrix[self.I + i, j] += self._theta_function(j)

        return r_matrix

    def _make_prob_matrix(self):
        prob_matrix = np.zeros((self.I * 2 + 1, self.N))
        prob_matrix[self.I, 0] = 1

        for j in range(self.N - 1):
            # j：列番号
            print("j", j)

            for i in range(min([j + 1, self.I + 1])):
                if i == 0:
                    """
                    中心線は別の処理
                    タイプAにまとめると、中心線を二重に処理することになるので。
                    """
                    # i: 中心線からの距離
                    # 各点の確率を計算
                    eta = (self.a * i * self.delta_x * self.delta_t) ** 2 + self.sigma ** 2 * self.delta_t
                    up_prob = (eta - self.a * i * self.delta_x ** 2 * self.delta_t) / (2 * self.delta_x ** 2)
                    mid_prob = 1 - eta / self.delta_x ** 2
                    down_prob = (eta + self.a * i * self.delta_x ** 2 * self.delta_t) / (2 * self.delta_x ** 2)

                    # 格子中心線付近
                    prob_matrix[I - 1, j + 1] += prob_matrix[I, j] * up_prob
                    prob_matrix[I, j + 1] += prob_matrix[I, j] * mid_prob
                    prob_matrix[I + 1, j + 1] += prob_matrix[I, j] * down_prob
                    
                elif i != I:
                    """
                    タイプA
                    """
                    # i: 中心線からの距離
                    # 各点の確率を計算
                    eta = (self.a * i * self.delta_x * self.delta_t) ** 2 + self.sigma ** 2 * self.delta_t
                    up_prob = (eta - self.a * i * self.delta_x ** 2 * self.delta_t) / (2 * self.delta_x ** 2)
                    mid_prob = 1 - eta / self.delta_x ** 2
                    down_prob = (eta + self.a * i * self.delta_x ** 2 * self.delta_t) / (2 * self.delta_x ** 2)
                    # 格子上側
                    prob_matrix[I - i - 1, j + 1] += prob_matrix[I - i, j] * up_prob
                    prob_matrix[I - i, j + 1] += prob_matrix[I - i, j] * mid_prob
                    prob_matrix[I - i + 1, j + 1] += prob_matrix[I - i, j] * down_prob

                    # 格子下側
                    prob_matrix[I + i - 1, j + 1] += prob_matrix[I + i, j] * up_prob
                    prob_matrix[I + i, j + 1] += prob_matrix[I + i, j] * mid_prob
                    prob_matrix[I + i + 1, j + 1] += prob_matrix[I + i, j] * down_prob

                    print("prob", up_prob + mid_prob + down_prob)
                else:
                    """
                    タイプC
                    """ 
                    # 格子上側
                    mid_prob = 7/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - 3 * self.a * i * self.delta_t) / 2
                    down_prob = -1/3 - self.a ** 2 * i ** 2 * self.delta_t ** 2 + 2 * self.a * i * self.delta_t
                    down_2_prob = 1/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - self.a * i * self.delta_t) / 2

                    prob_matrix[I - i, j + 1] += prob_matrix[I - i, j] * mid_prob
                    prob_matrix[I - i + 1, j + 1] += prob_matrix[I - i, j] * down_prob
                    prob_matrix[I - i + 2, j + 1] += prob_matrix[I - i, j] * down_2_prob


                    print("prob",up_prob + mid_prob + down_prob)
                    """
                    タイプB
                    """ 
                    # 格子下側
                    up_2_prob = 1/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - self.a * i * self.delta_t) / 2
                    down_prob = -1/3 - self.a ** 2 * i ** 2 * self.delta_t ** 2 + 2 * self.a * i * self.delta_t
                    down_2_prob = 7/6 + (self.a ** 2 * i ** 2 * self.delta_t ** 2 - 3 * self.a * i * self.delta_t) / 2

                    prob_matrix[I + i - 2, j + 1] += prob_matrix[I + i, j] * up_2_prob
                    prob_matrix[I + i - 1, j + 1] += prob_matrix[I + i, j] * up_prob
                    prob_matrix[I + i, j + 1] += prob_matrix[I + i, j] * mid_prob

        
                    print("prob",up_prob + mid_prob + down_prob)
        return prob_matrix

In [16]:
instance = HullWhite_grid(T, a, sigma, phi, r_0, N)

j 0
j 1
prob 1.0
j 2
prob 1.0
prob 1.0
j 3
prob 1.0
prob 1.0
prob 1.0
j 4
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 5
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 6
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 7
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 8
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 9
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 10
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 11
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 12
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 13
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
j 14
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0
prob 1.0

In [17]:
r = instance.r_matrix
x = instance.X_matrix
prob = instance.prob_matrix

In [21]:
np.sum(prob, axis = 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

instance

In [22]:
instance.calc_discount_bond_price()

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [19]:
prob[:,15]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.31879163e-15, 3.37750307e-13,
       2.19131521e-11, 8.39503193e-10, 2.12467573e-08, 3.76856883e-07,
       4.85311124e-06, 4.64099234e-05, 3.34458286e-04, 1.83371720e-03,
       7.69218597e-03, 2.47551639e-02, 6.11253613e-02, 1.15495431e-01,
       1.65883024e-01, 1.78367382e-01, 1.55440215e-01, 1.19999553e-01,
       8.09758827e-02, 4.74970554e-02, 2.41496704e-02, 1.06168908e-02,
       4.02054369e-03, 1.30342807e-03, 3.58222552e-04, 8.22155600e-05,
       1.53996957e-05, 2.27087019e-06, 2.48263516e-07, 1.79614306e-08,
      

In [14]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9
